In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
# Load the Excel file (after extraction if zipped)
energy_data_raw = pd.read_excel("Capstone 2025 Project- Utility Data.xlsx", header=None)

In [4]:
# Step 1: Set proper headers from row 5 (index 4)
headers = energy_data_raw.iloc[4].tolist()
energy_data = energy_data_raw.iloc[5:].copy()
energy_data.columns = headers

# Step 2: Clean and convert relevant columns
energy_data['Start\nDate'] = pd.to_datetime(energy_data['Start\nDate'], errors='coerce')
energy_data['End\nDate'] = pd.to_datetime(energy_data['End\nDate'], errors='coerce')
energy_data['Billing\nPeriod'] = pd.to_datetime(energy_data['Billing\nPeriod'], format='%Y/%b', errors='coerce')
energy_data['Use'] = pd.to_numeric(energy_data['Use'], errors='coerce')
energy_data['Days'] = pd.to_numeric(energy_data['Days'], errors='coerce')

# Step 3: Sort the entire dataset by Building Name, Start Date, End Date
energy_data_sorted = energy_data.sort_values(by=['Building Name', 'Start\nDate', 'End\nDate'])

# Step 4: Reset index if needed (optional)
energy_data_sorted = energy_data_sorted.reset_index(drop=True)

# View the first few sorted rows
print(energy_data_sorted.head(10))

   CAAN Building Name Meter\nCode Commodity\nCode Billing\nPeriod Start\nDate  \
0  6358    64 DEGREES       E1600        ELECTRIC      2011-01-01  2010-12-31   
1  6358    64 DEGREES       K1600      NATURALGAS      2011-01-01  2010-12-31   
2  6358    64 DEGREES       W1600           WATER      2011-01-01  2010-12-31   
3  6358    64 DEGREES       E1600        ELECTRIC      2011-02-01  2011-01-31   
4  6358    64 DEGREES       K1600      NATURALGAS      2011-02-01  2011-01-31   
5  6358    64 DEGREES       W1600           WATER      2011-02-01  2011-01-31   
6  6358    64 DEGREES       E1600        ELECTRIC      2011-03-01  2011-02-28   
7  6358    64 DEGREES       K1600      NATURALGAS      2011-03-01  2011-02-28   
8  6358    64 DEGREES       W1600           WATER      2011-03-01  2011-02-28   
9  6358    64 DEGREES       E1600        ELECTRIC      2011-04-01  2011-03-31   

   End\nDate  Days        Use Use Unit  
0 2011-01-31    31  71200.000      KWH  
1 2011-01-31    31   1636.

In [5]:
# Find all unique building names
unique_buildings = energy_data_sorted['Building Name'].unique()

# Print the list of unique building names
print("Unique Building Names:\n")
for building in unique_buildings:
    print(building)

# Also print the number of unique buildings
print("\nTotal number of unique buildings:", len(unique_buildings))


Unique Building Names:

64 DEGREES
ALLERGY & IMMUNOLOGY
ALTMAN CLINICAL & TRANSLATIONAL RESEARCH INSTITUTE
APPLIED PHYSICS & MATHEMATICS
ATHENA PARKING
BASEBALL FIELD
BIOLOGY BUILDING
BIOLOGY FIELD STATION - LABORATORY
BIOMEDICAL RESEARCH FACILITY II
BIOMEDICAL SCIENCES BUILDING ADDITION
BIOMEDICAL SCIENCES BUILDING+BSB ADDITION
BIRCH AQUARIUM at SCRIPPS
BONNER HALL
CAMPUS SERVICES COMPLEX - BLDG A
CAMPUS SERVICES COMPLEX - BLDG B
CAMPUS SERVICES COMPLEX - BLDG C
CAMPUS SERVICES COMPLEX - BLDG D & BLDG H
CAMPUS SERVICES COMPLEX SERVICES FLEET
CAMPUS SERVICES COMPLEX SHOPS
CANYON VISTA ADMINISTRATIVE DINING FACILITY
CANYONVIEW AQUATIC & CLIMBING WALL FAC+ADMIN BLDG
CAREER SERVICES CENTER
CELLULAR & MOLECULAR MEDICINE EAST
CENTER FOR COASTAL STUDIES
CENTER FOR NOVEL THERAPEUTICS
CENTER HALL
CENTER for MAGNETIC RECORDING RESEARCH
CENTER for NEURAL CIRCUITS & BEHAVIOR
CENTRAL RESEARCH SERVICES FACILITY (CAGE WASH)
CHANCELLOR COMPLEX
CHE CAFE
COAST APARTMENTS TOTAL SF & MAIN METERS
COGNITIV

In [6]:
# Check the total number of null values in each column
null_counts = energy_data_sorted.isnull().sum()

# Print the result
print("Null values in each column:\n")
print(null_counts)

# Optionally, show percentage of missing values for better clarity
null_percentage = (energy_data_sorted.isnull().sum() / len(energy_data_sorted)) * 100
print("\nPercentage of missing values in each column:\n")

null_percentage

Null values in each column:

CAAN                  0
Building Name         0
Meter\nCode           0
Commodity\nCode       0
Billing\nPeriod       0
Start\nDate           0
End\nDate             0
Days                  0
Use                1476
Use Unit              0
dtype: int64

Percentage of missing values in each column:



CAAN               0.000000
Building Name      0.000000
Meter\nCode        0.000000
Commodity\nCode    0.000000
Billing\nPeriod    0.000000
Start\nDate        0.000000
End\nDate          0.000000
Days               0.000000
Use                0.994134
Use Unit           0.000000
dtype: float64

In [7]:
# Filter rows where 'Use' is missing
missing_use_rows = energy_data_sorted[energy_data_sorted['Use'].isnull()]

# Display the rows
print("Rows where 'Use' is missing:\n")
print(missing_use_rows)

Rows where 'Use' is missing:

        CAAN Building Name Meter\nCode Commodity\nCode Billing\nPeriod  \
24      6358    64 DEGREES       E1600        ELECTRIC      2012-01-01   
25      6358    64 DEGREES       E1600        ELECTRIC      2012-02-01   
26      6358    64 DEGREES       E1600        ELECTRIC      2012-03-01   
27      6358    64 DEGREES       E1600        ELECTRIC      2012-04-01   
28      6358    64 DEGREES       E1600        ELECTRIC      2012-05-01   
...      ...           ...         ...             ...             ...   
147276  6361     YORK HALL       E1152        ELECTRIC      2012-05-01   
147277  6361     YORK HALL       E1153        ELECTRIC      2012-05-01   
147278  6361     YORK HALL       E1151        ELECTRIC      2012-06-01   
147279  6361     YORK HALL       E1152        ELECTRIC      2012-06-01   
147280  6361     YORK HALL       E1153        ELECTRIC      2012-06-01   

       Start\nDate  End\nDate  Days  Use Use Unit  
24      2011-12-31 2012-01-31

In [13]:
# Step 1: Create 'Start Month' for easier comparison (normalize to first of month)
energy_data_sorted['Start_Month'] = energy_data_sorted['Start\nDate'].values.astype('datetime64[M]')

# Step 2: Shift 'Start Month' within each Building Name + Meter Code group
energy_data_sorted['Previous Month'] = energy_data_sorted.groupby(['Building Name', 'Meter\nCode'])['Start_Month'].shift(1)

# Step 3: Calculate difference in months
energy_data_sorted['Month Difference'] = ((energy_data_sorted['Start_Month'] - energy_data_sorted['Previous Month']) / pd.Timedelta(days=30)).round()

# Step 4: Find where Month Difference > 1 (missing months)
missing_months = energy_data_sorted[energy_data_sorted['Month Difference'] > 1]

# Step 5: Display
print("Rows where months are missing:\n")
print(missing_months[['Building Name', 'Meter\nCode', 'Previous Month', 'Start_Month', 'Month Difference']])

Rows where months are missing:

       Building Name Meter\nCode Previous Month Start_Month  Month Difference
26        64 DEGREES       E1600     2012-01-01  2012-03-01               2.0
31        64 DEGREES       K1600     2011-05-01  2012-07-01              14.0
32        64 DEGREES       W1600     2011-05-01  2012-07-01              14.0
63        64 DEGREES       E1600     2013-04-01  2013-07-01               3.0
64        64 DEGREES       W1600     2013-04-01  2013-07-01               3.0
...              ...         ...            ...         ...               ...
147706     YORK HALL       E1153     2017-01-01  2017-03-01               2.0
147707     YORK HALL       H1151     2017-01-01  2017-03-01               2.0
147708     YORK HALL       K1150     2017-01-01  2017-03-01               2.0
147709     YORK HALL       W1150     2017-01-01  2017-03-01               2.0
147710     YORK HALL      W1152I     2017-01-01  2017-03-01               2.0

[6001 rows x 5 columns]
